In [3]:
!kaggle competitions download -c titanic --path ./data
!unzip -o ./data/titanic.zip -d ./data

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./data/titanic.zip
  inflating: ./data/gender_submission.csv  
  inflating: ./data/test.csv         
  inflating: ./data/train.csv        
